28丨EM聚类（上）：如何将一份菜等分给两个人？
====
今天我来带你学习 EM 聚类。EM 的英文是 Expectation Maximization，所以 EM 算法也叫最大期望算法。

我们先看一个简单的场景：假设你炒了一份菜，想要把它平均分到两个碟子里，该怎么分？

很少有人用称对菜进行称重，再计算一半的分量进行平分。大部分人的方法是先分一部分到碟子 A 中，然后再把剩余的分到碟子 B 中，再来观察碟子 A 和 B 里的菜是否一样多，哪个多就匀一些到少的那个碟子里，然后再观察碟子 A 和 B 里的是否一样多……整个过程一直重复下去，直到份量不发生变化为止。

你能从这个例子中看到三个主要的步骤：初始化参数、观察预期、重新估计。首先是先给每个碟子初始化一些菜量，然后再观察预期，这两个步骤实际上就是期望步骤（Expectation）。如果结果存在偏差就需要重新估计参数，这个就是最大化步骤（Maximization）。这两个步骤加起来也就是 EM 算法的过程。
<img src="./images/28-01.jpg">

EM 算法的工作原理
====
说到 EM 算法，我们先来看一个概念“最大似然”，英文是 Maximum Likelihood，Likelihood 代表可能性，所以最大似然也就是最大可能性的意思。

什么是最大似然呢？举个例子，有一男一女两个同学，现在要对他俩进行身高的比较，谁会更高呢？根据我们的经验，相同年龄下男性的平均身高比女性的高一些，所以男同学高的可能性会很大。这里运用的就是**最大似然的概念**。

最大似然估计是什么呢？它指的就是一件事情已经发生了，然后反推更有**可能是什么因素造成的**。还是用一男一女比较身高为例，假设有一个人比另一个人高，反推他可能是男性。**最大似然估计是一种通过已知结果，估计参数的方法。**

那么 EM 算法是什么？它和最大似然估计又有什么关系呢？EM 算法是一种**求解最大似然估计的方法**，通过观测样本，来找出样本的模型参数。

再回过来看下开头我给你举的分菜的这个例子，实际上最终我们想要的是碟子 A 和碟子 B 中菜的份量，你可以把它们理解为想要求得的**模型参数**。然后我们通过 EM 算法中的 E 步来进行观察，然后通过 M 步来进行调整 A 和 B 的参数，最后让碟子 A 和碟子 B 的参数不再发生变化为止。

实际我们遇到的问题，比分菜复杂。我再给你举个一个投掷硬币的例子，假设我们有 A 和 B 两枚硬币，我们做了 5 组实验，每组实验投掷 10 次，然后统计出现正面的次数，实验结果如下：
<img src="./images/28-02.png">

投掷硬币这个过程中存在隐含的数据，即我们事先并不知道每次投掷的硬币是 A 还是 B。假设我们知道这个隐含的数据，并将它完善，可以得到下面的结果：
<img src="./images/28-03.png">

我们现在想要求得硬币 A 和 B 出现正面次数的概率，可以直接求得：
<img src="./images/28-04.png">

**而实际情况是我不知道每次投掷的硬币是 A 还是 B**，那么如何求得硬币 A 和硬币 B 出现正面的概率呢？

这里就需要采用 EM 算法的思想。
1. 初始化参数。我们假设硬币 A 和 B 的正面概率（随机指定）是θA=0.5 和θB=0.9。2. 
2. 计算期望值。假设实验 1 投掷的是硬币 A，那么正面次数为 5 的概率为：
<img src="./images/28-05.png">

    公式中的 C(10,5) 代表的是 10 个里面取 5 个的组合方式，也就是排列组合公式，0.5 的 5 次方乘以 0.5 的 5 次方代表的是其中一次为 5 次为正面，5 次为反面的概率，然后再乘以 C(10,5) 等于正面次数为 5 的概率。

    假设实验 1 是投掷的硬币 B ，那么正面次数为 5 的概率为：
    <img src="./images/28-06.png">
    所以实验 1 更有可能投掷的是硬币 A。

    然后我们对实验 2~5 重复上面的计算过程，可以推理出来硬币顺序应该是{A，A，B，B，A}。

    这个过程实际上是通过假设的参数来估计未知参数，即“每次投掷是哪枚硬币”。
    
3. 通过猜测的结果{A, A, B, B, A}来完善初始化的参数θA 和θB。

    然后一直重复第二步和第三步，直到参数不再发生变化。

简单总结下上面的步骤，你能看出 EM 算法中的 E 步骤就是通过**旧的参数来计算隐藏变量**。然后在 M 步骤中，通过得到的隐藏变量的结果来重新估计参数。直到参数不再发生变化，得到我们想要的结果。

EM 聚类的工作原理
====
上面你能看到 EM 算法最直接的应用就是求参数估计。如果我们把潜在类别当做隐藏变量，样本看做观察值，就可以把聚类问题转化为参数估计问题。这也就是 EM 聚类的原理。

相比于 K-Means 算法，EM 聚类更加灵活，比如下面这两种情况，K-Means 会得到下面的聚类结果。
<img src="./images/28-07.jpg">

因为 K-Means 是通过距离来区分样本之间的差别的，且每个样本在计算的时候只能属于一个分类，称之为是**硬聚类算法**。而 EM 聚类在求解的过程中，实际上每个样本都有一定的概率和每个聚类相关，叫做**软聚类算法**。

你可以把 EM 算法理解成为是一个**框架**，在这个框架中可以采用不同的模型来用 EM 进行求解。常用的 EM 聚类有 **GMM 高斯混合模型**和 **HMM 隐马尔科夫模型**。GMM（高斯混合模型）聚类就是 EM 聚类的一种。比如上面这两个图，可以采用 GMM 来进行聚类。

和 K-Means 一样，我们事先知道聚**类的个数**，但是不知道每个样本分别属于哪一类。通常，我们可以假设样本是符合**高斯分布的（也就是正态分布）**。每个高斯分布都属于这个模型的组成部分（component），要分成 K 类就相当于是 K 个组成部分。这样我们可以先初始化每个组成部分的高斯分布的参数，然后再看来每个样本是属于哪个组成部分。这也就是 E 步骤。

再通过得到的这些隐含变量结果，反过来求每个组成部分高斯分布的参数，即 M 步骤。反复 EM 步骤，直到每个组成部分的高斯分布参数不变为止。

这样也就相当于将样本按照 GMM 模型进行了 EM 聚类。
<img src="./images/28-08.jpg">

总结
====
EM 算法相当于一个框架，你可以采用不同的模型来进行聚类，比如 GMM（高斯混合模型），或者 HMM（隐马尔科夫模型）来进行聚类。GMM 是通过概率密度来进行聚类，聚成的类符合高斯分布（正态分布）。而 HMM 用到了马尔可夫过程，在这个过程中，我们通过状态转移矩阵来计算状态转移的概率。HMM 在自然语言处理和语音识别领域中有广泛的应用。

在 EM 这个框架中，E 步骤相当于是通过初始化的参数来估计隐含变量。M 步骤就是通过隐含变量反推来优化参数。最后通过 EM 步骤的迭代得到模型参数。

在这个过程里用到的一些数学公式这节课不进行展开。你需要重点理解 EM 算法的原理。通过上面举的炒菜的例子，你可以知道 EM 算法是一个**不断观察和调整**的过程。

通过求硬币正面概率的例子，你可以理解如何通过初始化参数来求隐含数据的过程，以及再通过求得的隐含数据来优化参数。

通过上面 GMM 图像聚类的例子，你可以知道很多 K-Means 解决不了的问题，EM 聚类是可以解决的。在 EM 框架中，我们将**潜在类别当做隐藏变量**，样本看做观察值，把**聚类问题**转化为**参数估计问题**，最终把样本进行聚类。
<img src="./images/28-09.png">

思考题
====
1. 你能用自己的话说一下 EM 算法的原理吗？
2. EM 聚类和 K-Means 聚类的相同和不同之处又有哪些？



29丨EM聚类（下）：用EM算法对王者荣耀英雄进行划分
====
今天我来带你进行 EM 的实战。上节课，我讲了 EM 算法的原理，EM 算法相当于一个聚类框架，里面有不同的聚类模型，比如 GMM 高斯混合模型，或者 HMM 隐马尔科夫模型。其中你需要理解的是 EM 的两个步骤，E 步和 M 步：E 步相当于通过初始化的参数来估计隐含变量，M 步是通过隐含变量来反推优化参数。最后通过 EM 步骤的迭代得到最终的模型参数。

今天我们进行 EM 算法的实战，你需要思考的是：
* 如何使用 EM 算法工具完成聚类？
* 什么情况下使用聚类算法？我们用聚类算法的任务目标是什么？
* 面对王者荣耀的英雄数据，EM 算法能帮助我们分析出什么？

如何使用 EM 工具包
====
在 Python 中有第三方的 EM 算法工具包。由于 EM 算法是一个聚类框架，所以你**需要明确你要用的具体算法**，比如是采用 **GMM 高斯混合模型**，还是 **HMM 隐马尔科夫模型**。这节课我们主要讲解 GMM 的使用，在使用前你需要引入工具包：
```python
from sklearn.mixture import GaussianMixture
```
我们看下如何在 sklearn 中创建 GMM 聚类。

首先我们使用 **gmm = GaussianMixture(n_components=1, covariance_type=‘full’, max_iter=100)** 来创建 GMM 聚类，其中有几个比较主要的参数（GMM 类的构造参数比较多，我筛选了一些主要的进行讲解），我分别来讲解下：
1. n_components：即高斯混合模型的个数，也就是我们要聚类的个数，默认值为 1。如果你不指定 n_components，最终的聚类结果都会为同一个值。
2. covariance_type：代表协方差类型。一个高斯混合模型的分布是由**均值向量**和**协方差矩阵**决定的，所以协方差的类型也代表了不同的**高斯混合模型的特征**。协方差类型有 4 种取值：
    * covariance_type=full，代表完全协方差，也就是元素都不为 0；
    * covariance_type=tied，代表相同的完全协方差；
    * covariance_type=diag，代表对角协方差，也就是对角不为 0，其余为 0；
    * covariance_type=spherical，代表球面协方差，非对角为 0，对角完全相同，呈现球面的特性。
3. max_iter：代表最大迭代次数，EM 算法是由 E 步和 M 步迭代求得最终的模型参数，这里可以指定最大迭代次数，默认值为 100。

创建完 GMM 聚类器之后，我们就可以传入数据让它进行迭代拟合。

我们使用 fit 函数，传入样本特征矩阵，模型会自动生成聚类器，然后使用 prediction=gmm.predict(data) 来对数据进行聚类，传入你想进行聚类的数据，可以得到聚类结果 prediction。

你能看出来拟合训练和预测可以传入相同的特征矩阵，这是因为聚类是无监督学习，你不需要事先指定聚类的结果，也无法基于先验的结果经验来进行学习。只要在训练过程中传入特征值矩阵，机器就会按照特征值矩阵生成聚类器，然后就可以使用这个聚类器进行聚类了。

如何用 EM 算法对王者荣耀数据进行聚类
====
了解了 GMM 聚类工具之后，我们看下如何对王者荣耀的英雄数据进行聚类。

首先我们知道聚类的原理是“人以群分，物以类聚”。通过聚类算法把特征值相近的数据归为一类，不同类之间的差异较大，这样就可以对原始数据进行**降维**。通过分成几个组（簇），来研究每个组之间的特性。或者我们也可以把组（簇）的数量适当提升，这样就可以找到可以互相替换的英雄，比如你的对手选择了你擅长的英雄之后，你可以选择另一个英雄作为备选。

我们先看下数据长什么样子：
<img src="./images/29-01.png">
这里我们收集了 69 名英雄的 20 个特征属性，这些属性分别是最大生命、生命成长、初始生命、最大法力、法力成长、初始法力、最高物攻、物攻成长、初始物攻、最大物防、物防成长、初始物防、最大每 5 秒回血、每 5 秒回血成长、初始每 5 秒回血、最大每 5 秒回蓝、每 5 秒回蓝成长、初始每 5 秒回蓝、最大攻速和攻击范围等。

现在我们需要对王者荣耀的英雄数据进行聚类，我们先设定项目的执行流程：
<img src="./images/29-02.jpg">
1. 首先我们需要加载数据源；
2. 在准备阶段，我们需要对数据进行探索，包括采用数据可视化技术，让我们对英雄属性以及这些属性之间的关系理解更加深刻，然后对数据质量进行评估，是否进行数据清洗，最后进行特征选择方便后续的聚类算法；
3. 聚类阶段：选择适合的聚类模型，这里我们采用 GMM 高斯混合模型进行聚类，并输出聚类结果，对结果进行分析。

按照上面的步骤，我们来编写下代码。完整的代码如下：
```python
# -*- coding: utf-8 -*-
import pandas as pd
import csv
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.mixture import GaussianMixture
from sklearn.preprocessing import StandardScaler
 
# 数据加载，避免中文乱码问题
data_ori = pd.read_csv('./heros7.csv', encoding = 'gb18030')
features = [u'最大生命',u'生命成长',u'初始生命',u'最大法力', u'法力成长',u'初始法力',u'最高物攻',u'物攻成长',u'初始物攻',u'最大物防',u'物防成长',u'初始物防', u'最大每5秒回血', u'每5秒回血成长', u'初始每5秒回血', u'最大每5秒回蓝', u'每5秒回蓝成长', u'初始每5秒回蓝', u'最大攻速', u'攻击范围']
data = data_ori[features]
 
# 对英雄属性之间的关系进行可视化分析
# 设置plt正确显示中文
plt.rcParams['font.sans-serif']=['SimHei'] #用来正常显示中文标签
plt.rcParams['axes.unicode_minus']=False #用来正常显示负号
# 用热力图呈现features_mean字段之间的相关性
corr = data[features].corr()
plt.figure(figsize=(14,14))
# annot=True显示每个方格的数据
sns.heatmap(corr, annot=True)
plt.show()
 
# 相关性大的属性保留一个，因此可以对属性进行降维
features_remain = [u'最大生命', u'初始生命', u'最大法力', u'最高物攻', u'初始物攻', u'最大物防', u'初始物防', u'最大每5秒回血', u'最大每5秒回蓝', u'初始每5秒回蓝', u'最大攻速', u'攻击范围']
data = data_ori[features_remain]
data[u'最大攻速'] = data[u'最大攻速'].apply(lambda x: float(x.strip('%'))/100)
data[u'攻击范围']=data[u'攻击范围'].map({'远程':1,'近战':0})
# 采用Z-Score规范化数据，保证每个特征维度的数据均值为0，方差为1
ss = StandardScaler()
data = ss.fit_transform(data)
# 构造GMM聚类
gmm = GaussianMixture(n_components=30, covariance_type='full')
gmm.fit(data)
# 训练数据
prediction = gmm.predict(data)
print(prediction)
# 将分组结果输出到CSV文件中
data_ori.insert(0, '分组', prediction)
data_ori.to_csv('./hero_out.csv', index=False, sep=',')
```
运行结果如下：
<img src="./images/29-03.png">

```python
[28 14  8  9  5  5 15  8  3 14 18 14  9  7 16 18 13  3  5  4 19 12  4 12
 12 12  4 17 24  2  7  2  2 24  2  2 24  6 20 22 22 24 24  2  2 22 14 20
 14 24 26 29 27 25 25 28 11  1 23  5 11  0 10 28 21 29 29 29 17]
```
同时你也能看到输出的聚类结果文件 hero_out.csv（它保存在你本地运行的文件夹里，程序会自动输出这个文件，你可以自己看下）。我来简单讲解下程序的几个模块。

## 關於引用包
首先我们会用 DataFrame 数据结构来保存读取的数据，最后的聚类结果会写入到 CSV 文件中，因此会用到 pandas 和 CSV 工具包。另外我们需要对数据进行可视化，采用热力图展现属性之间的相关性，这里会用到 matplotlib.pyplot 和 seaborn 工具包。在数据规范化中我们使用到了 Z-Score 规范化，用到了 StandardScaler 类，最后我们还会用到 sklearn 中的 GaussianMixture 类进行聚类。
## 數據可視化的探索
你能看到我们将 20 个英雄属性之间的关系用热力图呈现了出来，中间的数字代表两个属性之间的关系系数，最大值为 1，代表完全正相关，关系系数越大代表相关性越大。从图中你能看出来“最大生命”“生命成长”和“初始生命”这三个属性的相关性大，我们只需要保留一个属性即可。同理我们也可以对其他相关性大的属性进行筛选，保留一个。你在代码中可以看到，我用 features_remain 数组保留了特征选择的属性，这样就将原本的 20 个属性降维到了 13 个属性。
## 關於數據規範化
我们能看到“最大攻速”这个属性值是百分数，不适合做矩阵运算，因此我们需要将百分数转化为小数。我们也看到“攻击范围”这个字段的取值为远程或者近战，也不适合矩阵运算，我们将取值做个映射，用 1 代表远程，0 代表近战。然后采用 Z-Score 规范化，对特征矩阵进行规范化。
## 在聚类阶段
我们采用了 GMM 高斯混合模型，并将结果输出到 CSV 文件中。

这里我将输出的结果截取了一段（设置聚类个数为 30）：
<img src="./images/29-04.png">
第一列代表的是分组（簇），我们能看到张飞、程咬金分到了一组，牛魔、白起是一组，老夫子自己是一组，达摩、典韦是一组。聚类的特点是相同类别之间的属性值相近，不同类别的属性值差异大。因此如果你擅长用典韦这个英雄，不防试试达摩这个英雄。同样你也可以在张飞和程咬金中进行切换。这样就算你的英雄被别人选中了，你依然可以有备选的英雄可以使用。

总结
====
今天我带你一起做了 EM 聚类的实战，具体使用的是 GMM 高斯混合模型。从整个流程中可以看出，我们需要经过数据加载、数据探索、数据可视化、特征选择、GMM 聚类和结果分析等环节。

聚类和分类不一样，聚类是无监督的学习方式，也就是我们没有实际的结果可以进行比对，所以聚类的结果评估不像分类准确率一样直观，那么有没有聚类结果的评估方式呢？这里我们可以采用 Calinski-Harabaz 指标，代码如下：
```python
from sklearn.metrics import calinski_harabaz_score
print(calinski_harabaz_score(data, prediction))
```
指标分数越高，代表聚类效果越好，也就是**相同类中的差异性小**，不同类之间的差异性大。当然具体聚类的结果含义，我们**需要人工来分析**，也就是当这些数据被分成不同的类别之后，具体每个类表代表的含义。

另外聚类算法也可以作为其他数据挖掘算法的预处理阶段，这样我们就可以将数据进行降维了。
<img src="./images/29-05.png">

思考题
====
1. 针对王者荣耀的英雄数据集，我进行了特征选择，实际上王者荣耀的英雄数量并不多，我们可以省略特征选择这个阶段，你不妨用**全部的特征值矩阵进行聚类训练**，来看下聚类得到的结果。
2. 第二个问题是，依然用王者荣耀英雄数据集，在聚类个数为 3 以及聚类个数为 30 的情况下，请你使用 GMM 高斯混合模型对数据集进行聚类，并得出 Calinski_Harabaz 分数。